In [2]:
import netket as nk
from utils import *
N = 10

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
"""
mainTEBD.py
---------------------------------------------------------------------
Script file for initializing the Hamiltonian and MPS tensors before passing to
the TEBD routine.

by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 06/2020
"""

# preamble
import numpy as np
from dotebd import doTEBD
from ncon import ncon

""" Example 1: XX model """
m = 10
# set bond dimensions and simulation options
chi = 16  # bond dimension
tau = 0.1  # timestep

numiter = 500  # number of timesteps
evotype = "imag"  # real or imaginary time evolution
E0 = -4 / np.pi  # specify exact ground energy (if known)
midsteps = int(1 / tau)  # timesteps between MPS re-orthogonalization

# define Hamiltonian (quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
sZ = np.array([[1, 0], [0, -1]])

#ham even
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
#ham odd
hamBA = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
print(hamAB.shape)
spin_1_hilbert = nk.hilbert.Spin(1,2)
hamAB = create_local_ham(0,m, 2, spin_1_hilbert,t=0, l_cons = 30, return_type= "dense").reshape(3,3,3,3)
hamBA = create_local_ham(0,m, 2, spin_1_hilbert,t=0, l_cons = 30, return_type= "dense").reshape(3,3,3,3)
# initialize tensors
d = hamAB.shape[0]
sAB = np.ones(chi) / np.sqrt(chi)
sBA = np.ones(chi) / np.sqrt(chi)
A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)

""" Imaginary time evolution with TEBD """
# run TEBD routine
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep
tau = 0.01
numiter = 2000
midsteps = 100
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep and increased bond dim
chi = 32
tau = 0.001
numiter = 20000
midsteps = 1000
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# compare with exact results
energyMPS = np.real(0.5 * ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]]) +
                    0.5 * ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]]))
enErr = abs(energyMPS - E0)
print('Final results => Bond dim: %d, Energy: %f, Energy Error: %e' %
      (chi, energyMPS, enErr))

(2, 2, 2, 2)
iteration: 0 of 500, chi: 16, t-step: 0.100000, energy: -5.882862, energy error: -4.609622e+00
iteration: 10 of 500, chi: 16, t-step: 0.100000, energy: -9.712908, energy error: -8.439668e+00
iteration: 20 of 500, chi: 16, t-step: 0.100000, energy: -9.749308, energy error: -8.476068e+00
iteration: 30 of 500, chi: 14, t-step: 0.100000, energy: -9.749987, energy error: -8.476748e+00
iteration: 40 of 500, chi: 10, t-step: 0.100000, energy: -9.750000, energy error: -8.476760e+00
iteration: 50 of 500, chi: 8, t-step: 0.100000, energy: -9.750000, energy error: -8.476760e+00
iteration: 60 of 500, chi: 6, t-step: 0.100000, energy: -9.750000, energy error: -8.476760e+00
iteration: 70 of 500, chi: 5, t-step: 0.100000, energy: -9.750000, energy error: -8.476760e+00
iteration: 80 of 500, chi: 4, t-step: 0.100000, energy: -9.750000, energy error: -8.476760e+00
iteration: 90 of 500, chi: 3, t-step: 0.100000, energy: -9.750000, energy error: -8.476760e+00
iteration: 100 of 500, chi: 3, t-

KeyboardInterrupt: 

array([[[[ 5.25,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  5.25,  0.  ],
         [ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  5.25],
         [ 0.  , -1.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ]]],


       [[[ 0.  ,  0.  ,  0.  ],
         [ 0.25,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.25,  0.  ],
         [ 0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.25],
         [ 0.  ,  0.  ,  0.  ]]],


       [[[ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ],
         [ 7.25,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  7.25,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  7.25]]]])